<a href="https://colab.research.google.com/github/mittalsharad/NLP/blob/main/HuggingFace/%20Transformer_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Pipeline

In [ ]:
#!pip install transformers

In [3]:
import transformers
from transformers import pipeline

In [4]:
classifier = pipeline("sentiment-analysis")
classifier([
    "I've been waiting for a HuggingFace course my whole life.", 
    "I hate this so much!",
    "I am so mad right now",
    "You are the worst person I have ever met",
    "I love to walk in garden",
])

[{'label': 'POSITIVE', 'score': 0.9598047733306885},
 {'label': 'NEGATIVE', 'score': 0.9994558095932007},
 {'label': 'NEGATIVE', 'score': 0.999092698097229},
 {'label': 'NEGATIVE', 'score': 0.9997233748435974},
 {'label': 'POSITIVE', 'score': 0.9998172521591187}]

# Decode the Pipeline

## Preprocessing with a tokenizer

In [5]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [6]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.", 
    "I hate this so much!",
    "I am so mad right now",
    "You are the worst person I have ever met",
    "I love to walk in garden",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0],
        [  101,  1045,  2572,  2061,  5506,  2157,  2085,   102,     0,     0,
             0,     0,     0,     0,     0,     0],
        [  101,  2017,  2024,  1996,  5409,  2711,  1045,  2031,  2412,  2777,
           102,     0,     0,     0,     0,     0],
        [  101,  1045,  2293,  2000,  3328,  1999,  3871,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


# Going Through the Model

In [7]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['pre_classifier.bias', 'classifier.weight', 'classifier.bias', 'pre_classifier.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([5, 16, 768])


In [10]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)
print(outputs.logits.shape)

torch.Size([5, 2])


In [11]:
print(outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464],
        [ 3.9045, -3.0996],
        [ 4.5590, -3.6335],
        [-4.1607,  4.4465]], grad_fn=<AddmmBackward>)


In [12]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04],
        [9.9909e-01, 9.0735e-04],
        [9.9972e-01, 2.7665e-04],
        [1.8275e-04, 9.9982e-01]], grad_fn=<SoftmaxBackward>)


In [13]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

So, we can say that:
- 1 : Positive
- 2: Negative
- 3: Negative 
- 4: Negative
- 5: Positive